In [4]:
#OpenAPI를 활용해서 2019~2022년의 모든 미세먼지경보 데이터를 가져오는 것이 목표!.

# 모듈 import
import requests  # 파이썬에서 웹서버에 접속하여 HTML 을 받아오기 위해 사용

### xml을 DataFrame으로 변환하기 위한 모듈 import ###
import pandas as pd
import xml.etree.ElementTree as et #기본적으로 xml을 처리하는 라이브러리.
import bs4                         #웹페이지에서 크롤링할 때 사용
from lxml import html              #html 데이터를 다루기 위한 라이브러리
from urllib.parse import urlencode, quote_plus, unquote 
#urlencode : HTTP GET이나 POST 등 query parameter를 encoding
#quote : 인코딩 수행. 다만, 파이썬에서 URL 처리 시 한글이 포함되면 깨질 수 있어서 해결책으로 quote_plus 사용
#unquote : url 디코딩 수행



#20XX년의 미세먼지경보 데이터를 가져오는 함수
def getmisae(year):
    
    # 값을 초기화시켜줍니다.
    lenofdf_temp = 100 # 사실 이 부분이 좀 다른데, 페이지를 계속 넘기다가 이제 페이지에 암것도 없으면 멈추는 코드를 짜려고 따로 초기화했어
                       # df_temp의 값이 0이면 이제 아래있는 while문 탈출하게 하려고 한 거야!
                       # 2023년 2024년에는 미세먼지 경보가 2000번 3000번 울릴 수도 있다고 생각해서 이렇게 했어!
                       # 뭐가 더 빠른지는 잘 모르겠긴 해.. 서버요청하고 응답 한 번만 받도록 하는 게 더 빠를 수도 있겠다 생각하긴 했는데
                       # 이게 그래도 더 정확할 것 같아서 while문 반복으로 진행했어!
    n = 0 # n은 페이지수입니다.
    df = pd.DataFrame()
    url = 'https://apis.data.go.kr/B552584/UlfptcaAlarmInqireSvc/getUlfptcaAlarmInfo' #url 입력
    
    while lenofdf_temp > 0:
        n += 1
        params = {'serviceKey' : 'rsa4maEy0uEr46KTaSi77ShfqpdyUNFgGsU2GI2t8Dg8X4qZ3bSN7rZr4wuufa6eLExlPkasrip8eeeUH7jMdw==',
              'pageNo': n, 'numOfRows':'100', 'year': year, 'returnType':'xml'}
        # URL 뒤에 붙는 파라미터들을 설정합니다. pageNo는 while문 속에서 증가하며,
        # year은 getmisae에서 설정된 값으로 탐색합니다.
        
        response = requests.get(url, params = params)
        # 여기서 API를 요청하고 response에 저장합니다.

        #xml 내용
        content = response.text #태그를 떼어냄

        #bs4 사용해서 item 태그를 분리
        xml_obj = bs4.BeautifulSoup(content, 'lxml-xml')
        rows = xml_obj.findAll('item')
        # 잘 분리되었는지 확인하고 싶다면
        # print(rows)
        # 를 쓰면 됩니다.

        # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
        row_list = []   # 행값
        name_list = []  # 열이름값
        value_list = [] # 데이터값

        #xml 안의 데이터 수집
        for i in range(0, len(rows)):
            columns = rows[i].find_all()
            #첫번째 행 데이터 수집
            for j in range(0, len(columns)):
                if i == 0:
                    #column name 값 저장
                    name_list.append(columns[j].name)
                #컬럼의 각 데이터 값 저장
                value_list.append(columns[j].text)
            #각 행의 value 값 전체 저장
            row_list.append(value_list)
            #데이터 리스트 값 초기화
            value_list=[]

        #xml 값 DataFrame으로 만들기
        df_temp = pd.DataFrame(row_list, columns = name_list)
        df = pd.concat([df, df_temp], axis = 0)
        lenofdf_temp = len(df_temp)
        # 이번 페이지의 df 길이를 저장합니다. 돌아가 while문에사 이 값이 0이라면 멈춥니다.
    
    return df

In [5]:
df_list = [2019, 2020, 2021, 2022]
# 확인하고 싶은 연도를 설정합니다.
df = pd.DataFrame()
# df를 초기화해줍니다.

for i in df_list:
    df_temp = getmisae(i)
    df = pd.concat([df, df_temp], axis = 0)

# for문을 돌면서 df_list의 값을 getmisae 함수에 집어넣습니다.
# 계속 df에 붙입니다.
    
df

,clearVal,sn,districtName,dataDate,issueVal,issueTime,clearDate,issueDate,moveName,clearTime,issueGbn,itemCode
0,34,2498,세종,2019-12-24,84,15:00,2019-12-26,2019-12-24,세종권역,16:00,주의보,PM25
1,34,2497,충북,2019-12-24,77,15:00,2019-12-26,2019-12-24,중부권역,21:00,주의보,PM25
2,33,2496,충남,2019-12-24,87,14:00,2019-12-26,2019-12-24,북부권역,21:00,주의보,PM25
3,60,2494,경남,2019-12-17,195,13:00,2019-12-17,2019-12-17,거제권역,14:00,주의보,PM10
4,4,2495,경남,2019-12-17,143,13:00,2019-12-17,2019-12-17,거제권역,15:00,주의보,PM25
...,...,...,...,...,...,...,...,...,...,...,...,...
78,33,5,인천,2022-01-08,86,23:00,2022-01-11,2022-01-08,강화권역,02:00,주의보,PM25
79,27,11,인천,2022-01-08,83,00:00,2022-01-10,2022-01-09,동남부권역,20:00,주의보,PM25
80,23,1,서울,2022-01-08,82,22:00,2022-01-10,2022-01-08,서울권역,21:00,주의보,PM25
81,30,2,경기,2022-01-08,81,22:00,2022-01-11,2022-01-08,북부권,03:00,주의보,PM25


In [6]:
df = df.reset_index(drop=True)
df.columns = ['해제농도', '관리번호', '지역명', '발령일', '발령농도', '발령시간', '해제일',
              '발령일', '권역명' ,'해제시간', '경보단계', '미세먼지 항목 구분']

# 인덱스를 다시 설정하고 이름을 이쁘게 달아줍니다.

df

,해제농도,관리번호,지역명,발령일,발령농도,발령시간,해제일,발령일,권역명,해제시간,경보단계,미세먼지 항목 구분
0,34,2498,세종,2019-12-24,84,15:00,2019-12-26,2019-12-24,세종권역,16:00,주의보,PM25
1,34,2497,충북,2019-12-24,77,15:00,2019-12-26,2019-12-24,중부권역,21:00,주의보,PM25
2,33,2496,충남,2019-12-24,87,14:00,2019-12-26,2019-12-24,북부권역,21:00,주의보,PM25
3,60,2494,경남,2019-12-17,195,13:00,2019-12-17,2019-12-17,거제권역,14:00,주의보,PM10
4,4,2495,경남,2019-12-17,143,13:00,2019-12-17,2019-12-17,거제권역,15:00,주의보,PM25
...,...,...,...,...,...,...,...,...,...,...,...,...
2346,33,5,인천,2022-01-08,86,23:00,2022-01-11,2022-01-08,강화권역,02:00,주의보,PM25
2347,27,11,인천,2022-01-08,83,00:00,2022-01-10,2022-01-09,동남부권역,20:00,주의보,PM25
2348,23,1,서울,2022-01-08,82,22:00,2022-01-10,2022-01-08,서울권역,21:00,주의보,PM25
2349,30,2,경기,2022-01-08,81,22:00,2022-01-11,2022-01-08,북부권,03:00,주의보,PM25
